S metaheuristike
- lokalna pretraga
- simulirano kaljenje
- VNS - metoda promenljivih okolina
- tabu pretraga
- iterativana lokalna pretraga
- ...

P metaheuristike

In [1]:
# UFLP

# 10 korisnika
# 5 bolnica
# c
# f

# solution = [True, False, False, True, False]

In [2]:
import random

In [3]:
def isFeasible(solution):
    for resource in solution:
        if resource:
            return True
    return False

In [4]:
def initialize(numResources):
    solution = [random.random() < 0.25 for _ in range(numResources)]
    if not isFeasible(solution):
        chosenResource = random.randrange(0, numResources)
        solution[chosenResource] = True
    
    return solution

In [5]:
def calcSolutionValue(solution, cost, fixedCost):
    value = 0
    numUsers = len(cost)
    numResources = len(fixedCost)
    
    used = [False for _ in range(numResources)]
#     used = {}
    
    for i in range(numUsers):
        minCost = float('inf')
        resourceUsed = -1
        for j in range(numResources):
            if solution[j] and cost[i][j] < minCost:
                minCost = cost[i][j]
                resourceUsed = j
        value += minCost
        used[resourceUsed] = True
#         used.add(resourceUsed)
        
    for i in range(numResources):
        if used[i]:
            value += fixedCost[i]
    
    solution = used
    
    return value

In [8]:
# calcSolutionValue([True, False, False], cost, fixedCost)

In [9]:
def invert(solution):
    chosenResource = random.randrange(len(solution))
    solution[chosenResource] = not solution[chosenResource]
    if isFeasible(solution):
        return chosenResource
    else:
        solution[chosenResource] = not solution[chosenResource]
        return -1

In [10]:
def localSearch(cost, fixedCost, iters):
    solution = initialize(numResources=len(fixedCost))
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue
    
    for i in range(iters):
#         newSolution = izmeni malo solution
        resourceInverted = invert(solution)
        if resourceInverted < 0:
            continue
        newValue = calcSolutionValue(solution, cost, fixedCost)
        
        if newValue < currValue:
            currValue = newValue
            if newValue < bestValue:
                bestValue = newValue
        else:
#             nastavi sa starim solution
            solution[resourceInverted] = not solution[resourceInverted]
            
    return bestValue

In [11]:
def readInput(filename):
    with open(filename, 'r') as f:
        numUsers, numResources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for _ in range(numUsers)]
        fixedCost = [int(x) for x in f.readline().split()]
        return cost, fixedCost

In [12]:
cost, fixedCost = readInput('uflp1.txt')
localSearch(cost, fixedCost, 100)

34

In [13]:
def simulatedAnnealing(cost, fixedCost, iters):
    solution = initialize(len(fixedCost))
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue
    
    for i in range(iters):
        invertedResource = invert(solution)
        if invertedResource < 0:
            continue
        newValue = calcSolutionValue(solution, cost, fixedCost)
        
        if newValue < currValue:
            currValue = newValue
            if newValue < bestValue:
                bestValue = newValue
        else:
            p = 1.0 / i ** 0.5
            q = random.uniform(0, 1)
            if p > q:
                currValue = newValue
            else:
                solution[invertedResource] = not solution[invertedResource]
    return bestValue

In [14]:
simulatedAnnealing(cost, fixedCost, 100)

34

In [15]:
# RVNS - reduced VNS
# solution = initialize()
# currValue = calcSolutionValue(solution)
# bestValue = currValue

# while not stop_condition():
#     k = 0
#     while k <= num_neiborhoods:
#         newSolution = getNeighbor(solution, k)
#         newValue = calcSolutionValue(newSolution)
#         if newValue < currValue:
#             currValue = newValue
#             solution = newSolution
#             break
#         else:
#             k += 1
            
# return bestValue

In [18]:
# BVNS - optimizuj newSolution nekom vrstom lokalnom pretrage
# GVNS - optimizuj newSolution VNS

In [ ]:
# [1,0,0,1]

In [21]:
def getNeighbor(solution, k):
    indices = range(len(solution))
    toInvert = random.sample(indices, k)
    for index in toInvert:
        solution[index] = not solution[index]
        
    if isFeasible(solution):
        return True, toInvert
    else:
        return False, toInvert

In [23]:
def restore(solution, invertedIndices):
    for resource in invertedIndices:
        solution[resource] = not solution[resource]        

In [24]:
def RVNS(cost, fixedCost, iters, max_k):
    solution = initialize(len(fixedCost))
    currValue = calcSolutionValue(solution, cost, fixedCost)
    bestValue = currValue
    
    for i in range(iters):
        k = 0
        while k <= max_k:
            success, invertedIndices = getNeighbor(solution, k)
            if not success:
                # vrati se na originalni solution
                restore(solution, invertedIndices)
                continue
            newValue = calcSolutionValue(solution, cost, fixedCost)
            if newValue < currValue:
                currValue = newValue
                if newValue < bestValue:
                    bestValue = newValue
                break
            else:
                k += 1
                restore(solution, invertedIndices)
    return bestValue            

In [28]:
RVNS(cost, fixedCost, 10, 2)

34